<a href="https://colab.research.google.com/github/ahmedlabib02/Nlp-project/blob/main/Nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone 1

In [95]:
import os 
import json
import re
import pandas as pd
from collections import Counter
import string
from transformers import BertTokenizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sana2\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

### Data preparation

#### Unzipping the "B Hodoo2" channel folder

In [25]:
import zipfile
zip_path = "./B Hodoo2-20250226T182935Z-001.zip"
extract_path = "./"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


#### Unzipping the "Kefaya ba2a" channel folder

In [26]:
import zipfile
zip_path = "./Kefaya Ba2a-20250226T183527Z-001.zip"
extract_path = "./"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

### Building dict for annotations and text

In [2]:
def standardize(title):
    
    title = title.replace("_", " ")
   
    title = title.lower()
    
    title = re.sub(r'[^\w\s]', '', title)
    
    # title = re.sub(r'\bبودكاست بهدوء مع كريم\b', '', title)
    # title = re.sub(r'\bجلسة\b', '', title)
    
    title = re.sub(r'\s+', ' ', title).strip()
    return title

In [4]:
def get_category_for_filename(target_filename, annotations_path):
    """
    Load annotations from annotations_path and try to match the
    canonicalized target_filename (without file extension) to the canonicalized
    title from each annotation. If a match is found, return the annotation's category.
    """
    with open(annotations_path, 'r', encoding='utf-8') as f_json:
        annotations = json.load(f_json)
    
    base_target = os.path.splitext(target_filename)[0]
    target_can = standardize(base_target)
    for entry in annotations:
        annotation_title = entry.get('title')
        if annotation_title:
            
            annotation_can = standardize(annotation_title)
            if annotation_can == target_can:
                # print("Matched annotation:", entry.get('title'))
                return entry.get('category')
    return None

In [5]:
def add_transcripts_from_folder(transcripts_dict, folder_path, channel):
    
    annotations_json = os.path.join(folder_path, 'annotations.json')
    raw_folder = os.path.join(folder_path, 'raw')
    for filename in os.listdir(raw_folder):
        full_path = os.path.join(raw_folder, filename)
        if os.path.isfile(full_path):
            with open(full_path, 'r', encoding='utf-8') as file:
                transcript = file.read()
                category = get_category_for_filename(filename, annotations_json)
            base_filename, _ = os.path.splitext(filename)
            transcripts_dict[base_filename] = (channel, category, transcript)
    return transcripts_dict

In [6]:
transcripts_dict = {}
first_folder_path = './B Hodoo2/'
second_folder_path = './Kefaya Ba2a/'

transcripts_dict = add_transcripts_from_folder(transcripts_dict, first_folder_path,  'B Hodoo2')
transcripts_dict = add_transcripts_from_folder(transcripts_dict, second_folder_path, 'Kefaya Ba2a')



### Creating a dataframe

In [79]:
data = [(title, info[0], info[1], info[2]) for title, info in transcripts_dict.items()]
df = pd.DataFrame(data, columns=['title', 'channel', 'category', 'transcript'])
print(df[0:1])


                                               title   channel   category  \
0  أكبر كذبة في التاريخ _ وهم الانتاجية! - _ بودك...  B Hodoo2  Education   

                                          transcript  
0  النهارده الفيديو بتاعنا على\nالانتاجيه لان الن...  


### Analysis

In [22]:
category_counts = Counter(df['category'])
print(category_counts)

Counter({'Education': 26, 'People & Blogs': 17, 'Comedy': 4})


**Since there are some entries without categories or an annotation file, we just remove them.**

In [80]:
df = df[df['category']!= None]

**Exploring dataset size**

In [33]:
print(df.shape)

(47, 4)


### Cleaning the data

#### Removing numbers, non-Arabic letters and extra spaces

In [81]:
def clean_text(text):
    punctuations = string.punctuation + "؟،؛«»…ـ"  
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)  
    text = re.sub(r'[0-9A-Za-z]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
file_path = "B Hodoo2/raw\لماذا نيأس؟ عن فلسفة المثابرة _ بودكاست بهدوء مع كريم _ جلسة 20.txt" 
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()
print(clean_text(text))

In [82]:
df["transcript"] = df["transcript"].apply(clean_text)
print(df.head())

                                               title   channel   category  \
0  أكبر كذبة في التاريخ _ وهم الانتاجية! - _ بودك...  B Hodoo2  Education   
1  إزاي ضاعفت دخلي بدون شغل إضافي! - عن الرزق ووس...  B Hodoo2  Education   
2  اخطر محتوى في العالم! _ بودكاست بهدوء مع كريم ...  B Hodoo2  Education   
3  العيد والنكد - ليه مش بنبسط في العيد؟ _ بودكاس...  B Hodoo2  Education   
4  انا فقير _ اصحابي كلهم أغنى مني! _ بودكاست بهد...  B Hodoo2  Education   

                                          transcript  
0  النهارده الفيديو بتاعنا على الانتاجيه لان النه...  
1  انا النهارده جاي اقوللك ازاي تزود من دخلك الما...  
2  النهارده انا جاي اتكلم على واحده من اخطر الظوا...  
3  بصراحه انا موضوع ان احنا مش عارفين نتبسط في ال...  
4  مساء الفل الفيديو ده عن الرزق لو انت واحد من ا...  


### Tokenization and Stop word removal

In [83]:
print(df['transcript'])

0     النهارده الفيديو بتاعنا على الانتاجيه لان النه...
1     انا النهارده جاي اقوللك ازاي تزود من دخلك الما...
2     النهارده انا جاي اتكلم على واحده من اخطر الظوا...
3     بصراحه انا موضوع ان احنا مش عارفين نتبسط في ال...
4     مساء الفل الفيديو ده عن الرزق لو انت واحد من ا...
5     الفيديو ده ثقيل جدا على قلبي وللاسف الفيديو ده...
6     عندنا في مصر لما بيبقى في حد الدنيا مخبطه معاه...
7     مافيش حد فينا الا وفي حياته نسبه من القلق سواء...
8     يدخل البيت فبيقول لها وصفي لي الحرامي بوليس بق...
9     النهارده انا جاي اكلمك في موضوع في منتهى الخطو...
10    الحلقه دي بقى بالذات انا دايما يا جماعه زي ما ...
11    من حوالي سبع سنين واحده كتبت لي في التعليقات ي...
12    السلام عليكم ورحمه انت ازاي تدخل عليا بوكاس كد...
13    اول يا جماعه لما بتقول الله هو من فعل بذلك الن...
14    طبعا كالعاده احنا اتعودنا ان اي فيديو يبدا لاز...
15    النهارده انا بهدي الحلقه دي لنفسي اولا والله ب...
16    ينفع اشارككم حاجه بدون مثاليه عاليه ومبالغات ب...
17    وكالعاده مستمرين في التمرد على الغوريزم ول

In [84]:
def tokenize(df):
    tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
    df['transcript'] = df['transcript'].apply(lambda x: tokenizer.tokenize(str(x)))

In [85]:
tokenize(df)



In [88]:
print(df[[ 'transcript']])

                                           transcript
0   [ال, ##نها, ##رد, ##ه, ال, ##فيد, ##يو, ب, ##ت...
1   [ان, ##ا, ال, ##نها, ##رد, ##ه, جا, ##ي, ا, ##...
2   [ال, ##نها, ##رد, ##ه, ان, ##ا, جا, ##ي, ا, ##...
3   [ب, ##صر, ##اح, ##ه, ان, ##ا, موضوع, ان, ا, ##...
4   [م, ##ساء, ال, ##فل, ال, ##فيد, ##يو, ده, عن, ...
5   [ال, ##فيد, ##يو, ده, ث, ##قيل, جدا, على, قلب,...
6   [عند, ##نا, في, مصر, لما, بي, ##بقى, في, حد, ا...
7   [ما, ##في, ##ش, حد, في, ##نا, ال, ##ا, وفي, حي...
8   [يد, ##خل, ال, ##بيت, ف, ##بي, ##قول, لها, وصف...
9   [ال, ##نها, ##رد, ##ه, ان, ##ا, جا, ##ي, ا, ##...
10  [ال, ##حل, ##قه, دي, ب, ##قى, با, ##ل, ##ذا, #...
11  [من, حوالي, سب, ##ع, سن, ##ين, واحد, ##ه, كتب,...
12  [السلام, علي, ##كم, و, ##رح, ##مه, ان, ##ت, از...
13  [اول, يا, ج, ##ماع, ##ه, لما, ب, ##ت, ##قول, ا...
14  [ط, ##بع, ##ا, ك, ##ال, ##عاد, ##ه, ا, ##حن, #...
15  [ال, ##نها, ##رد, ##ه, ان, ##ا, به, ##دي, ال, ...
16  [ي, ##نف, ##ع, اش, ##ارك, ##كم, ح, ##اج, ##ه, ...
17  [و, ##كا, ##ل, ##عاد, ##

In [96]:
arabic_stopwords = set(stopwords.words('arabic'))
def remove_stopwords(tokens):
    return [token for token in tokens if token not in arabic_stopwords]

df['transcript'] = df['transcript'].apply(remove_stopwords)


In [97]:
df['transcript']

0     [ال, ##نها, ##رد, ##ه, ال, ##فيد, ##يو, ##تا, ...
1     [ان, ##ا, ال, ##نها, ##رد, ##ه, جا, ##ي, ##قول...
2     [ال, ##نها, ##رد, ##ه, ان, ##ا, جا, ##ي, ##ت, ...
3     [##صر, ##اح, ##ه, ان, ##ا, موضوع, ان, ##حن, ##...
4     [##ساء, ال, ##فل, ال, ##فيد, ##يو, ده, ال, ##ر...
5     [ال, ##فيد, ##يو, ده, ##قيل, جدا, قلب, ##ي, ##...
6     [##نا, مصر, ##بقى, حد, ال, ##دن, ##يا, ##خ, ##...
7     [##في, ##ش, حد, ##نا, ال, ##ا, وفي, حياته, ##س...
8     [يد, ##خل, ال, ##بيت, ##بي, ##قول, وصف, ##ي, ا...
9     [ال, ##نها, ##رد, ##ه, ان, ##ا, جا, ##ي, ##كل,...
10    [ال, ##حل, ##قه, دي, ##قى, با, ##ل, ##ذا, ##ت,...
11    [حوالي, سب, ##ع, سن, ##ين, ##ه, كتب, ##ت, ال, ...
12    [السلام, علي, ##كم, ##رح, ##مه, ان, ##ت, از, #...
13    [اول, ##ماع, ##ه, ##ت, ##قول, الله, فعل, بذلك,...
14    [##بع, ##ا, ##ال, ##عاد, ##ه, ##حن, ##ا, ##ت, ...
15    [ال, ##نها, ##رد, ##ه, ان, ##ا, ##دي, ال, ##حل...
16    [##نف, ##ع, اش, ##ارك, ##كم, ##اج, ##ه, بدون, ...
17    [##كا, ##ل, ##عاد, ##ه, ##ست, ##مر, ##ين, 